Import libraries

In [2]:
import sys
sys.path.insert(0,'..')

import numpy as np
import pandas as pd
import mxnet as mx
from mxnet import nd
from mxnet import autograd
from mxnet import init
from mxnet import gluon
from mxnet.gluon import nn
from mxnet.gluon import data as gluon_data
from mxnet.gluon import loss as gluon_loss

In [3]:
ctx = mx.cpu()

In [4]:
train_data = pd.read_csv("Data/train.csv")
test_data = pd.read_csv("Data/test.csv")

In [77]:
train_labels = nd.array(train_data.iloc[:,0])
train_features = nd.array(train_data.iloc[:,1:785])

In [106]:
dataset = gluon.data.ArrayDataset(train_features,train_labels)
train_iter = gluon.data.DataLoader(dataset,batch_size,shuffle=True)

In [133]:
net = nn.Sequential()
net.add(nn.Dense(784,activation='relu'))
net.add(nn.Dense(300,activation='relu'))
net.add(nn.Dense(200,activation='relu'))
net.add(nn.Dense(100,activation='relu'))
net.add(nn.Dense(10))
net.initialize(init.Normal(sigma=0.01))

In [134]:
loss_function = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate' : 0.1} )

In [135]:
num_epochs = 10

In [136]:
train_loss, test_loss = [],[]

for epoch in range(1,num_epochs+1):
    for data,label in train_iter:
        with autograd.record():
            y_hat = net(data)
            loss_value = loss_function(y_hat,label).sum()
            
        loss_value.backward()
        trainer.step(batch_size)       
        
    print("epoch: ",epoch,"loss: ",loss_value.mean().asnumpy())

epoch:  1 loss:  [ 5.3581109]
epoch:  2 loss:  [ 0.8472088]
epoch:  3 loss:  [ 2.01559448]
epoch:  4 loss:  [ 0.19979014]
epoch:  5 loss:  [ 0.46389708]
epoch:  6 loss:  [ 0.44366637]
epoch:  7 loss:  [ 0.54955751]
epoch:  8 loss:  [ 0.49154451]
epoch:  9 loss:  [ 0.07531575]
epoch:  10 loss:  [ 0.00227232]


In [147]:
print(train_labels[1005])
max = net(train_features)[1005]
print(max.argmax)


[ 8.]
<NDArray 1 @cpu(0)>
<bound method NDArray.argmax of 
[ -6.36310673  -2.20514202   0.49976602   3.86541629  -6.99960661
  -1.0177542   -7.57814455  -5.83551884  23.210392     1.39221287]
<NDArray 10 @cpu(0)>>


In [20]:
train_data.shape
data1 = train_data.sample(frac=0.98,random_state=200)
data2 = train_data.drop(data1.index)
data2.shape

(840, 785)